In [19]:
from dotenv import load_dotenv 
from pathlib import Path
import os
load_dotenv(override=True)  # take environment variables from .env file

True

In [20]:
# -------------------------------
# Read environment variables
# -------------------------------
HITEC_EMAIL_DIR = os.getenv("GLOBAL_HITEC_EMAIL_DIR")
HITEC_EMAIL_JSON = os.getenv("GLOBAL_HITEC_EMAIL_JSON")
HITEC_EMAIL_EXCEL = os.getenv("GLOBAL_HITEC_EMAIL_EXCEL")
print(HITEC_EMAIL_DIR, HITEC_EMAIL_JSON)

GLOBAL_HITEC_OUTAGE_EMAILS GLOBAL_HITEC_OUTAGE_JSON


In [21]:
from langchain_openai import ChatOpenAI
import json
from datetime import datetime

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

PARSER_PROMPT = """
You are an expert outage email parser.
Extract the following fields and return ONLY valid JSON:

- partner_name
- outage_type
- issue_details
- current_status
- business_impact
- manual_processing
- outage_start_time
- outage_end_time
- duration_hours

Dates in emails are available in PST timezone. Use same timezone for storing dates
If missing, return null.
"""

def parse_email_with_llm(email_text: str):
    prompt = PARSER_PROMPT + "\n\nEmail:\n" + email_text
    result = llm.invoke(prompt).content.strip()

    # Extract valid JSON only
    start = result.find("{")
    end = result.rfind("}") + 1
    json_str = result[start:end]

    try:
        data = json.loads(json_str)
    except Exception:
        data = {"error": "LLM parsing failed", "raw": email_text[:500]}

    data["parsed_at"] = datetime.utcnow().isoformat()
    return data


In [26]:
import os
from pathlib import Path

def read_and_parse_email_folder(folder_path):
    folder = Path(folder_path)

    records = []

    for file in folder.glob("*.txt"):
        print(f"📩 Reading: {file.name}")
        text = file.read_text(errors="ignore")
        parsed = parse_email_with_llm(text)
        parsed["source_file"] = file.name
        records.append(parsed)
        

    return records


In [23]:
import pandas as pd

def save_records_to_excel(records, excel_path="outages.xlsx"):
    df = pd.DataFrame(records)
    df.to_excel(excel_path, index=False)
    print(f"✅ Saved Excel file with {len(df)} rows → {excel_path}")
    return df


In [24]:
import json

def save_records_as_json(records, json_path="outages.json"):
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(records, f, indent=2)
    print(f"✅ Saved JSON file → {json_path}")


In [27]:
if __name__ == "__main__":
    EMAIL_DIR = HITEC_EMAIL_DIR  # folder containing .txt email files

    print("📥 Reading and parsing emails...")
    records = read_and_parse_email_folder(EMAIL_DIR)

    print("\n📊 Saving to Excel...")
    df = save_records_to_excel(records, "outages.xlsx")

    # (Optional) also save JSON
    save_records_as_json(records, "outages.json")

    print("\n🎉 Completed!")


📥 Reading and parsing emails...
📩 Reading: outage_summary_001.txt
📩 Reading: outage_summary_002.txt
📩 Reading: outage_summary_003.txt
📩 Reading: outage_summary_004.txt
📩 Reading: outage_summary_005.txt
📩 Reading: outage_summary_006.txt
📩 Reading: outage_summary_007.txt
📩 Reading: outage_summary_008.txt
📩 Reading: outage_summary_009.txt
📩 Reading: outage_summary_010.txt
📩 Reading: outage_summary_011.txt
📩 Reading: outage_summary_012.txt
📩 Reading: outage_summary_013.txt
📩 Reading: outage_summary_014.txt
📩 Reading: outage_summary_015.txt
📩 Reading: outage_summary_016.txt
📩 Reading: outage_summary_017.txt
📩 Reading: outage_summary_018.txt
📩 Reading: outage_summary_019.txt
📩 Reading: outage_summary_020.txt
📩 Reading: outage_summary_021.txt
📩 Reading: outage_summary_022.txt
📩 Reading: outage_summary_023.txt
📩 Reading: outage_summary_024.txt
📩 Reading: outage_summary_025.txt
📩 Reading: outage_summary_026.txt
📩 Reading: outage_summary_027.txt
📩 Reading: outage_summary_028.txt
📩 Reading: outag